<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Plan" data-toc-modified-id="Plan-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Plan</a></span></li><li><span><a href="#Code" data-toc-modified-id="Code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="#Import-modules-and-load-functions" data-toc-modified-id="Import-modules-and-load-functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Import modules and load functions</a></span></li><li><span><a href="#Get-dataverse-info" data-toc-modified-id="Get-dataverse-info-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get dataverse info</a></span></li><li><span><a href="#Get-aliases-of-any-sub-dataverses-in-the-given-dataverse" data-toc-modified-id="Get-aliases-of-any-sub-dataverses-in-the-given-dataverse-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Get aliases of any sub-dataverses in the given dataverse</a></span></li><li><span><a href="#Get-dataset-info" data-toc-modified-id="Get-dataset-info-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Get dataset info</a></span></li></ul></li></ul></div>

## Plan

For chosen dataverse:
    - Show dataverse info:
        - Show number of subdataverses, if any
        - Show whether or not the (sub)dataverse has a description or tagline
        - List metadatablocks of each (sub)dataverse
        - List the facets set for each (sub)dataverse
        - Verify that contact email address is valid for each (sub)dataverse
        (see https://medium.com/@arjunsinghy96/verify-emails-over-socks-proxy-using-python-5589cb75c405
        and https://github.com/Gardener-gg/email-verifier)
    - Show dataset info
        - Show number of datasets
        - Show date of first published dataset
        - Show date of most recently published or updated dataset
        - Show average age of datasets
        - Show average number of dataset versions
        - Metadata (of latest published version of each dataset):
            - Show average number of characters in the dataset descriptions
                - List datasets with fewer than a certain number of characters in their descriptions
            - List datasets with CC0 or Terms of Use metadata
                - Versus number of datasets with no CC0 or TOU metadata
                - List datasets with no CC0 or TOU metadata
            - Show number of files that have no description metadata
                - If a certain percentage of datasets have 1 or more files with no descriptions, list those datasets
            - Related publication metadata
                - Show number of datasets with related publication metadata
                    - List datasets with no related publication metadata
                - Show number of datasets with no PID in related publication metadata
                    - List datasets with no PID in related publication metadata
            - Show datasets that have no metadata for any non-citation metadatablocks enabled in the dataverse
        - Data
            - List count of each unique file format
            - Show number of datasets with no files
                - List datasets that have no files
            - Show number of datasets with 1 or more uningested tabular files
                - List datasets that contain 1 or more uningested tabular files
            - Show number of datasets with 1 or more restricted files
        - Contact emails
            - Get number of datasets that have a contact email address that's different email from the dataverse contact email address
            - Get unique list of contact email addresses and check if they're valid
            - Show any datasets that have no valid email addresses

## Code

### Import modules and load functions

In [2]:
from datetime import datetime, timezone
from functools import reduce
import numpy as np
import pandas as pd
import requests
import sys
import time


def improved_get(_dict, path, default=None):
    for key in path.split('.'):
        try:
            _dict = _dict[key]
        except KeyError:
            return default
    return _dict


def list_to_string(list):
    # Alphabetize list in case-insensitive way
    list = sorted(list, key=lambda s: s.casefold())
    # Change list to comma-separated string
    delimiter = ","
    string = delimiter.join(list)
    return string


def string_to_datetime(string):
    newDatetime = datetime.strptime(string, '%Y-%m-%dT%H:%M:%S%z')
    return newDatetime

# current_time = str.strftime('%Y.%m.%d_%H.%M.%S')
# current_time = datetime.strftime('%Y-%m-%dT%H:%M:%S%z')
# currentTime = datetime.now()
currentTime = datetime.now(timezone.utc)


### Get dataverse info

In [3]:
# Get dataverse server and alias from user - return error if there's no alias or if alias is the Root dataverse
server = 'https://demo.dataverse.org'
mainDataverseAlias = 'sefsef'
# server = 'https://dataverse.harvard.edu'
# mainDataverseAlias = 'nds'

repositoryMetadataBlocksApi = '%s/api/v1/metadatablocks' % (server)
response = requests.get(repositoryMetadataBlocksApi)
repositoryMetadataBlocks = response.json()

repositoryMetadataBlockNames = []
for repositoryMetadataBlock in repositoryMetadataBlocks['data']:
    repositoryMetadataBlockNames.append(repositoryMetadataBlock['name'])

In [4]:
# Get info from that dataverse: whether or not the dataverse has a description and/or tagline, metadatablocks enabled, facets enabled, validate contact email
dataverseInfoApi = '%s/api/dataverses/%s' % (server, mainDataverseAlias)
response = requests.get(dataverseInfoApi)
dataverseMetadata = response.json()

In [5]:
if dataverseMetadata['status'] == 'ERROR':
    print('No dataverse found. Is the dataverse published on Harvard Dataverse?')
elif dataverseMetadata['status'] == 'OK':
    dataverseDescription = improved_get(dataverseMetadata, 'data.description')
    if dataverseDescription is not None:
        print('Dataverse has a description')
    else:
        print('Dataverse has no description')

    tagline = improved_get(dataverseMetadata, 'data.theme.tagline')
    if tagline is not None:
        print('Dataverse has a tagline')
    else:
        print('Dataverse has no tagline')

#     contactEmails = []
#     for contact in dataverseMetadata['data']['dataverseContacts']:
#         contactEmails.append(contact['contactEmail'])
#     print(contactEmails)

    dataverseFacetsApi = '%s/api/dataverses/%s/facets' % (server, mainDataverseAlias)
    response = requests.get(dataverseFacetsApi)
    dataverseFacets = response.json()
    facets = []
    for facet in dataverseFacets['data']:
        facets.append(facet)
    print('Number of search facets used: %s' % (len(facets)))    

#     # See if dataverse inherits its metadatablocks from its parent dataverse
#     metadatablocksInheritedApi = '%s/api/dataverses/%s/metadatablocks/isRoot' % (server, dataverseAlias)
#     response = requests.get(metadatablocksInheritedApi)
#     metadatablocksInherited = response.json()
#     print(metadatablocksInherited)
    
    # Get list of metadatablocks enabled in the dataverse
    dataverseMetadatablocksList = []
    dataverseMetadatablocksApi = '%s/api/dataverses/%s/metadatablocks' % (server, mainDataverseAlias)
    response = requests.get(dataverseMetadatablocksApi)
    dataverseMetadatablocks = response.json()
    for dataverseMetadatablock in dataverseMetadatablocks['data']:
        dataverseMetadatablock = dataverseMetadatablock['name']
        dataverseMetadatablocksList.append(dataverseMetadatablock)
    print('Number of metadatablocks enabled (in addition to Citation): %s' % (len(dataverseMetadatablocksList) - 1))


Dataverse has no description
Dataverse has no tagline
Number of search facets used: 6
Number of metadatablocks enabled (in addition to Citation): 5


### Get aliases of any sub-dataverses in the given dataverse

In [6]:
mainDataverseInfoApi = '%s/api/dataverses/%s' % (server, mainDataverseAlias)
response = requests.get(mainDataverseInfoApi)
data = response.json()
mainDataverseID = data['data']['id']

dataverseIDs = [mainDataverseID]
for dataverseID in dataverseIDs:

    sys.stdout.write('.')
    sys.stdout.flush()

    url = '%s/api/dataverses/%s/contents' % (server, dataverseID)

    response = requests.get(url)
    data = response.json()

    for i in data['data']:
        if i['type'] == 'dataverse':
            dataverseID = i['id']
            dataverseIDs.extend([dataverseID])

print('\n\nFound 1 dataverse and %s subdataverses' % (len(dataverseIDs) - 1))

...

Found 1 dataverse and 2 subdataverses


### Get dataset info

In [7]:
# Get PIDs of all published datasets in each of the dataverses
datasetPIDs = []
rowList = []
for dataverseID in dataverseIDs:
    getDataverseInfoApi = '%s/api/dataverses/%s' % (server, dataverseID)
    response = requests.get(getDataverseInfoApi)
    dataverseInfo = response.json()
    dataverseName = dataverseInfo['data']['name']
    dataverseAlias = dataverseInfo['data']['alias']

    getDataverseContentsApi = '%s/api/dataverses/%s/contents' % (server, dataverseID)
    response = requests.get(getDataverseContentsApi)
    dataverseContents = response.json()
    for item in dataverseContents['data']:
        if item['type'] == 'dataset':
            datasetPID = item['persistentUrl'].replace('https://doi.org/', 'doi:')
            datasetPIDs.append(datasetPID)
            
            newRow = {'datasetPID': datasetPID,
                  'dataverseName': dataverseName,
                  'dataverseUrl': '%s/dataverse/%s' % (server, dataverseAlias)
                 }
            rowList.append(dict(newRow))
            
            sys.stdout.write('.')
            sys.stdout.flush()
            
print('\nNumber of datasets: %s' % (len(datasetPIDs)))

datasetDataverseInfoDF = pd.DataFrame(rowList)


...Number of datasets: 3


In [7]:
# print(datasetPIDs)

Create a dataframe for dataset info: date of publication, the release date of the latest version, number of versions

_Getting this info can be slow. For example, getting the info of ~375 datasets might take 45 min_

In [15]:
# Create list of file types that Dataverse can convert to .tab files during ingest
uningestedFileTypes = ['application/x-rlang-transport', 'application/x-stata-13', 'application/x-spss-por',
                      'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'text/csv', 'text/tsv',
                      'application/x-spss-sav', 'text/comma-separated-values', 'application/x-stata',
                      'application/x-stata-14']

rowList = []
datasetCount = 0
for datasetPID in datasetPIDs:
    getAllVersionsApi = '%s/api/datasets/:persistentId/versions?persistentId=%s' % (server, datasetPID)
    response = requests.get(getAllVersionsApi)
    datasetVersions = response.json()
    
    # Get only datasets with metadata (exclude responses with no values in 'data' key, e.g. deaccessioned datasets)
    if datasetVersions['status'] == 'OK' and len(datasetVersions['data']) > 0:
        
        # Get metadata of latest version
        latestDatasetVersion = datasetVersions['data'][0]
        
        # Get index location of first dataset version
        firstVersion = len(datasetVersions['data']) - 1

        publicationDate = string_to_datetime(datasetVersions['data'][firstVersion]['releaseTime'])
        latestReleaseDate = string_to_datetime(latestDatasetVersion['releaseTime'])
        
        # Get age of dataset from today's date
        delta = currentTime - publicationDate
        ageOfDataset = delta.days
        
        # Get number of days since last update
        delta = currentTime - latestReleaseDate
        ageOfLastUpdate = delta.days
        if ageOfLastUpdate < 0:
            ageOfLastUpdate = 0
        
        # Get length of description text
        descriptionLength = 0
        
        for field in latestDatasetVersion['metadataBlocks']['citation']['fields']:
            if field['typeName'] == 'dsDescription':
                # "N/A" is the value assigned there was no description given (pre Dataverse 4)
                if len(field['value']) == 1 and field['value'][0]['dsDescriptionValue']['value'] == 'N/A':
                    descriptionLength = 0
                else:
                    for i in field['value']:
                        descriptionLength = descriptionLength + len(i['dsDescriptionValue']['value'])

        # See whether CC0 or Terms of Use metadata exists
        license = latestDatasetVersion.get('license', 'None')

        if 'termsOfUse' in latestDatasetVersion:
            termsOfUse = True
        else:
            termsOfUse = False
            
        if 'termsOfAccess' in latestDatasetVersion:
            termsOfAccess = True
        else:
            termsOfAccess = False

        if license != 'CC0' and termsOfUse == False:
            termsExist = False
        else:
            termsExist = True

        # Get info about related publication metadata
        relPubCount = 0
        relPubPIDCount = 0
        for field in latestDatasetVersion['metadataBlocks']['citation']['fields']:
            if field['typeName'] == 'publication':
                for value in field['value']:
                    relPubCount += 1
                    if 'publicationIDType' and 'publicationIDNumber' in value:
                        relPubPIDCount += 1
        
        # Show metadatablocks whose fields are used by the dataset
        usedMetadataBlocks = []
        for repositoryMetadataBlockName in repositoryMetadataBlockNames:
            try:
                fieldCount = len(latestDatasetVersion['metadataBlocks'][repositoryMetadataBlockName]['fields'])
                if fieldCount > 0:
                    usedMetadataBlocks.append(repositoryMetadataBlockName)
            except KeyError:
                usedMetadataBlocks = usedMetadataBlocks
        if len(usedMetadataBlocks) == 0:
            usedMetadataBlocks = ''
        else:
            usedMetadataBlocks = list_to_string(usedMetadataBlocks)
        
        # Get number of files
        numberOfFiles = len(latestDatasetVersion['files'])

        # Get file info
        noFileDescriptionCount = 0
        contentType = []
        ingestedTabFilesCount = 0
        uningestedTabFilesCount = 0
        restrictedFilesCount = 0
        fileTags = []
        for file in latestDatasetVersion['files']:            
            if 'description' in file:
                noFileDescriptionCount = noFileDescriptionCount
            else:
                noFileDescriptionCount += 1
            contentType.append(file['dataFile']['contentType'])
            if file['restricted'] == True:
                restrictedFilesCount += 1
            if file['dataFile']['contentType'] in uningestedFileTypes:
                uningestedTabFilesCount += 1
            if file['dataFile']['contentType'] == 'text/tab-separated-values':
                ingestedTabFilesCount += 1
            try:
                for tags in file['categories']:
                    fileTags.append(tags)
            except KeyError:
                fileTags = fileTags

        if len(fileTags) == 0:
            fileTagsExist = False
        else:
            fileTagsExist = True

        if len(contentType) == 0:
            uniqueContentTypes = 'NA'
        else:
            uniqueContentTypes = list_to_string(list(set(contentType)))

        # Create dictionary
        newRow = {'datasetPID': datasetPID,
                  'datasetPIDUrl' : datasetPID.replace('doi:', 'https://doi.org/'),
                  'numberOfVersions': len(datasetVersions['data']),
                  'numberOfMajorVersions': latestDatasetVersion['versionNumber'],
                  'publicationDate': publicationDate,
                  'latestReleaseDate': latestReleaseDate,
                  'ageOfDataset(Days)': ageOfDataset,
                  'ageOfLastUpdate(Days)': ageOfLastUpdate,
                  'descriptionLenth': descriptionLength,
                  'termsExist': termsExist,
                  'license': license,
                  'termsOfUseExists': termsOfUse,
                  'termsOfAccessExists': termsOfAccess,
                  'relPubCount': relPubCount,
                  'relPubPIDCount': relPubPIDCount,
                  'usedMetadataBlocks': usedMetadataBlocks,
                  'numberOfFiles': numberOfFiles,
                  'noFileDescriptionCount': noFileDescriptionCount,
                  'fileTagsExist': fileTagsExist,
                  'uniqueContentTypes': uniqueContentTypes,
                  'ingestedTabFilesCount': ingestedTabFilesCount,
                  'uningestedTabFilesCount': uningestedTabFilesCount,
                  'restrictedFilesCount': restrictedFilesCount
                 }
        rowList.append(dict(newRow))
        datasetCount += 1
        print('%s of %s (%s)' % (datasetCount, len(datasetPIDs), datasetPID), end='\r', flush=True)
        
if len(datasetPIDs) != datasetCount:
    print('The metadata of %s dataset(s) could not be retrieved' % (len(datasetPIDs) - datasetCount))


In [16]:
datasetInfoDF = pd.DataFrame(rowList)
datasetInfoDF

,datasetPID,datasetPIDUrl,numberOfVersions,numberOfMajorVersions,publicationDate,latestReleaseDate,ageOfDataset(Days),ageOfLastUpdate(Days),descriptionLenth,termsExist,...,relPubCount,relPubPIDCount,usedMetadataBlocks,numberOfFiles,noFileDescriptionCount,fileTagsExist,uniqueContentTypes,ingestedTabFilesCount,uningestedTabFilesCount,restrictedFilesCount
0,doi:10.70122/FK2/HZTO03,https://doi.org/10.70122/FK2/HZTO03,3,1,2020-08-04 19:48:40+00:00,2020-10-26 03:44:39+00:00,85,2,7,False,...,0,0,"citation,geospatial",0,0,False,NA,0,0,0
1,doi:10.70122/FK2/CMFTOD,https://doi.org/10.70122/FK2/CMFTOD,1,1,2020-10-14 20:07:47+00:00,2020-10-14 20:07:47+00:00,14,14,7,True,...,0,0,citation,2,2,False,image/jpeg,0,0,0
2,doi:10.70122/FK2/ZYUGHH,https://doi.org/10.70122/FK2/ZYUGHH,16,5,2020-09-17 16:08:53+00:00,2020-10-29 03:16:38+00:00,41,0,0,True,...,2,1,"astrophysics,biomedical,citation,geospatial,so...",3,2,True,"image/jpeg,image/png,text/tab-separated-values",1,0,0


In [17]:
dataframes = [datasetDataverseInfoDF, datasetInfoDF]

# For each dataframe, set the indexes (or the common columns across the dataframes to join on)
for dataframe in dataframes:
    dataframe.set_index(['datasetPID'], inplace=True)

# Merge both dataframes and save to the 'merged' variable
report = reduce(lambda left, right: left.join(right, how='outer'), dataframes)

# Reset index
report.reset_index(drop=False, inplace=True)

file = '%s_datasets.csv' % (mainDataverseAlias)
report.to_csv(file, index=False)


In [11]:
# # Show date of first published dataset
# publicationDates = df['publicationDate']
# firstPublicationDate = publicationDates.min()

# # Show date of most recently published or updated dataset
# latestReleaseDates = df['latestReleaseDate']
# lastReleaseDate = latestReleaseDates.min()

# # Show age of most recently published or updated dataset
# ageOfLastUpdates = df['ageOfLastUpdate']
# ageOfLastUpdate = ageOfLastUpdates.min()

# # Show average age of datasets
# ageOfDatasets = df['ageOfDataset(Days)']
# averageAge = ageOfDatasets.mean()

# # Show average number of dataset versions
# numberOfVersions = df['numberOfVersions']
# averageNumberOfVersions = numberOfVersions.mean()

# # Create list of datasets with fewer than a certain number of characters in their descriptions
# lowDescriptionCount = df[df['descriptionLenth'] < 20]
# lowDescriptionCount = lowDescriptionCount['datasetPID']


In [12]:
# List count of each unique file format